In [1]:
import numpy as np
import scipy as sp
import sklearn as sk
import matplotlib as mpl
mpl.use('Agg')
import matplotlib.pylab as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
import seaborn as sns
import statsmodels as sm
import statsmodels.api as sm
import pandas as pd
from patsy import dmatrix

C:\Users\ledes\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
data = pd.read_csv("train.csv")

In [3]:
#for문 진행 사항 확인(실제 사용은 하지 않음)
from tqdm import tqdm_notebook
#역지오코딩(위도, 경도를 주소로 전환)하기 위한 모듈 geopy 의 클래스 중 'nominatim'
#Nominatim 사용 이유 : 개수 제한 문제를 해결할 수 있으므로
from geopy.geocoders import Nominatim
#역지오코딩 속도 개선을 위한 다중 쓰레드 사용
import threading
#지속적으로 많은 requests가 날아가면 "too many requests" 확인 -> 랜덤 시간 적용
import random
import time

In [4]:
#Nominatim 패키지 초기에 생긴 '접근 권한 없음' 에러 해결
#에러의 원인은 모르나, 해당 함수를 사용하면 역지오코딩은 성공적으로 가능
from geopy.geocoders import Nominatim
from urllib.request import Request

def get_geolocator():
    geolocator = Nominatim()

    requester = geolocator.urlopen

    def requester_hack(req, **kwargs):
        req = Request(url=req, headers=geolocator.headers)
        return requester(req, **kwargs)

    geolocator.urlopen = requester_hack

    return geolocator

In [5]:
#raw dataset에서 위도 및 경도 데이터 출력
pickup_lat = data["pickup_latitude"]
pickup_lon = data["pickup_longitude"]    

In [6]:
#쓰레드 사용 시, 순서가 섞이지 않게 다수의 데이터 프레임 선언 - 추후 합칠 예정
post_code = pd.DataFrame(columns = ["post_code1"])
post_code2 = pd.DataFrame(columns = ["post_code2"])
post_code3 = pd.DataFrame(columns = ["post_code3"])

#### geopy package에 대한 설명은 구글에서 "python geopy documentation"을 통해 확인 가능

In [7]:
def reverse_geocode() :
    #오류 발생해도 계속 진행할 수 있게 try - except 구문 사용
    try : 
    #위도와 경도를 zip함수로 각각 a,b에 담음
        for a, b in list(zip(pickup_lat, pickup_lon))[ : 500000] :
            #reverse method에 접근하여 location 반환
            #reverse의 인자 중 time out을 설정하지 않으면 잦은 오류 발생
            #확실하진 않지만, 좌표값이 주소로 변환이 진행되는 중에 또 다른 요청을 넣으면 오류가 발생하는 듯
            #이를 해결하기 위해, selenium에서와 같이 데이터 처리가 진행되는 시간을 기다려주는 역할인 것 같다.

            location = get_geolocator().reverse(str(a) + ',' + str(b), timeout = 10)
            #location 중 address attribute에 접근하여 값 반환
            address = location.address
            post_code.loc[len(post_code)] = address[-31 : -26]
            #data 개수가 많아 1000개마다 데이터 개수를 확인하며 프린트
            if len(post_code) % 1000 == 0 :
                print("1번 dataset : ", len(post_code))
            n = random.randint(1, 3)
            time.sleep(n)
    except Exception as ex :
        print("1번 dataset의", len(post_code),"번째에서 오류 발생 : ", ex)
        pass
    
    
#이하 2개는 threading을 위한 함수. 위의 함수와 구조 동일
def reverse_geocode2() :
    try : 

        for a, b in list(zip(pickup_lat, pickup_lon))[500000 : 1000000] :
            location = get_geolocator().reverse(str(a) + ',' + str(b), timeout = 10)
            address = location.address
            post_code2.loc[len(post_code)] = address[-31 : -26]
            if len(post_code2) % 1000 == 0 :
                print("2번 dataset : ", len(post_code2))
            n = random.randint(1, 3)
            time.sleep(n)

    except Exception as ex :
        print("2번 dataset의",len(post_code2),"번째에서 오류 발생 : ", ex)
        pass
        
        
def reverse_geocode3() :
    
    try :
    
        for a, b in list(zip(pickup_lat, pickup_lon))[1000000 : ] :
            location = get_geolocator().reverse(str(a) + ',' + str(b), timeout = 11)
            address = location.address
            post_code3.loc[len(post_code)] = address[-31 : -26]
            if len(post_code3) % 1000 == 0 :
                print("3번 dataset : ", len(post_code3))
            n = random.randint(1, 3)
            time.sleep(n)

    
    except Exception as ex :
        print("3번 dataset의",len(post_code3),"번째에서 오류 발생 : ", ex)
        pass
        
        
        

In [8]:
th1 = threading.Thread(target = reverse_geocode)
#demon = False가 default인데, False일 경우 메인 쓰레드가 중단되어도 서브 쓰레드가 돌아감
#True로 반환하여 3개의 쓰레드가 모두 동시에 중단될 수 있게 해놓음(오류 발생 시 유용)
th1.demon  =True
th1.start()

th2 = threading.Thread(target = reverse_geocode2)
th2.demon = True
th2.start()

th3 = threading.Thread(target = reverse_geocode3)
th3.demon = True
th3.start()


In [9]:
print(len(post_code) + len(post_code2) + len(post_code3))

0
2번 dataset의3번 dataset의1번 dataset의   000   번째에서 오류 발생 : 번째에서 오류 발생 : 번째에서 오류 발생 :    HTTP Error 429: Too Many RequestsHTTP Error 429: Too Many RequestsHTTP Error 429: Too Many Requests


